# Ikeda for S175
Scope:
* S175 will be calculated in ScoresII:
   * S175 geometry in a *Seaway input file* will be parsed
   * *Seaway input file* will be converted to ScoresII
   * Draught need to be adjusted to 9.5 m

In [ ]:
# %load ../../imports.py
"""
These is the standard setup for the notebooks.
"""

%matplotlib inline
%load_ext autoreload
%autoreload 2

from jupyterthemes import jtplot
jtplot.style(theme='onedork', context='notebook', ticks=True, grid=False)

import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)
import numpy as np
import os
import matplotlib.pyplot as plt
#plt.style.use('paper')

#import data
import copy
from rolldecay.bis_system import BisSystem
from rolldecay import database
from mdldb.tables import Run

from sklearn.pipeline import Pipeline
from rolldecayestimators.transformers import CutTransformer, LowpassFilterDerivatorTransformer, ScaleFactorTransformer, OffsetTransformer
from rolldecayestimators.direct_estimator_cubic import EstimatorQuadraticB, EstimatorCubic
from rolldecayestimators.ikeda_estimator import IkedaQuadraticEstimator


In [ ]:
import re

## S175 geometry in a Seaway input file will be parsed

In [ ]:
with open('S175.seaway', mode='r') as file:
    lines = file.readlines()

In [ ]:
lines[0:3]

In [ ]:
regexp_line=re.compile('.*')
def read_line(line):
    result = regexp_line.search(line)
    if not result:
        raise ValueError('Cannot read line:%s'%lins)
    return result.group(0)

def read_line_values(line):
    s = read_line(line)
    regexp_parts = re.compile('(\S+)')
    return regexp_parts.findall(s)
        
    

In [ ]:
read_line(line=lines[0])

In [ ]:
seaway={}
seaway['version'] = read_line(lines[0])
seaway['text'] = read_line(lines[1])

In [ ]:
## Definition of Longitudinal Values:
values = read_line_values(lines[2])
keys=['DR','TR','RLPP','RLA']
for key,value in zip(keys,values):
    seaway[key]=float(value)
seaway

In [ ]:
seaway['NS']=int(read_line(lines[3]))
seaway['NS']

In [ ]:
seaway['DX']=np.array(read_line_values(lines[4]) + 
                      read_line_values(lines[5]) + 
                      read_line_values(lines[6]), dtype=float)
seaway['DX']

In [ ]:
seaway['KCON']=int(read_line(lines[7]))
seaway['KCON']  # KCON = 2: input-sequence is [Z(J,I),Y(J,I)],

In [ ]:
NS=seaway['NS']+1  # one extra?
empty=np.empty(shape=NS)*np.NaN
keys = ['SNR','NWL','SDIST']
for key in keys:
    seaway[key] = empty.copy()

index=8
seaway['Y']=[]
seaway['Z']=[]

for J in range(NS):
    values = read_line_values(lines[index])
    seaway['SNR'][J]=float(values[0])
    seaway['NWL'][J]=NWL=int(float(values[1]))
    seaway['SDIST'][J]=SNR=float(values[2])
    section_values=[]
    while(len(section_values)<(2*(NWL+1))):
        index+=1
        section_values+=read_line_values(lines[index])
    
    Y=np.empty(shape=NWL)*np.NaN
    Z=np.empty(shape=NWL)*np.NaN
    
    for I in range(NWL):
        i=2*I
        Z[I]=section_values[i]
        Y[I]=section_values[i+1]
        
    seaway['Y'].append(Y)
    seaway['Z'].append(Z)
    
    index+=1

## Seaway input file will be converted to ScoresII

## Draught need to be adjusted to 9.5 m